In [1]:
import pandas as pd
import numpy as np
import os.path

from keras.preprocessing.image import Iterator
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import load_model

import threading
lock = threading.Lock()

from BsonIterator import BSONIterator

Using TensorFlow backend.


In [2]:
METADATA_DIR = 'metadata'
DATA_DIR = '../../dataset'

train_offsets_df = pd.read_csv(os.path.join(METADATA_DIR,"train_offsets.csv"), index_col=0)
train_images_df = pd.read_csv(os.path.join(METADATA_DIR,"RandomSplit_Train_0.9_0.2.csv"), index_col=0)
val_images_df = pd.read_csv(os.path.join(METADATA_DIR,"RandomSplit_Val_0.9_0.2.csv"), index_col=0)

num_classes = 5270
num_train_images = len(train_images_df)
num_val_images = len(val_images_df)
batch_size = 128

# Create a generator for training and a generator for validation.
# Tip: use ImageDataGenerator for data augmentation and preprocessing.
assert os.path.exists(os.path.join(DATA_DIR, 'train.bson'))
train_bson_file = open(os.path.join(DATA_DIR, 'train.bson'), "rb")
train_datagen = ImageDataGenerator()
train_gen = BSONIterator(train_bson_file, train_images_df, train_offsets_df,
                         num_classes, train_datagen, lock,
                         batch_size=batch_size, shuffle=True)

val_datagen = ImageDataGenerator()
val_gen = BSONIterator(train_bson_file, val_images_df, train_offsets_df,
                       num_classes, val_datagen, lock,
                       batch_size=batch_size, shuffle=True)

/home/ankdesh/virtualenvs/cdiscount-py2.7/local/lib/python2.7/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Found 988516 images belonging to 5270 classes.
Found 242274 images belonging to 5270 classes.


In [3]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

# Save model Dir
MODEL_DIR = 'saved_models'

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer 
predictions = Dense(num_classes, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions, name= 'InceptionV3')

# compile the model (should be done *after* setting layers to non-trainable)
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

num_epochs = 10

# train the model on the new data for a few epochs
model.fit_generator(train_gen,
                    steps_per_epoch = num_train_images // batch_size,
                    epochs = num_epochs,
                    validation_data = val_gen,
                    validation_steps = num_val_images // batch_size,
                    workers = 4)
model.save(os.path.join(MODEL_DIR, model.name + '_fulltrain_epcs' + str(num_epochs) + '_RandomSplit_Train_0.9_0.2' + '.h5'))

Epoch 1/10
7722/7722 [==============================] - ETA: 0s - loss: 4.0082 - acc: 0.3245 - val_loss: 3.5546 - val_acc: 0.3681
Epoch 2/10
7722/7722 [==============================] - ETA: 0s - loss: 3.0219 - acc: 0.4368 - val_loss: 3.2161 - val_acc: 0.4211
Epoch 3/10
7722/7722 [==============================] - ETA: 0s - loss: 2.6545 - acc: 0.4797 - val_loss: 2.9618 - val_acc: 0.4648
Epoch 4/10
7722/7722 [==============================] - ETA: 0s - loss: 2.4036 - acc: 0.5100 - val_loss: 2.8870 - val_acc: 0.4790
Epoch 5/10
7722/7722 [==============================] - ETA: 0s - loss: 2.2130 - acc: 0.5343 - val_loss: 2.8422 - val_acc: 0.4910
Epoch 6/10
7722/7722 [==============================] - ETA: 0s - loss: 2.0455 - acc: 0.5566 - val_loss: 2.7512 - val_acc: 0.5077
Epoch 7/10
7722/7722 [==============================] - ETA: 0s - loss: 1.8997 - acc: 0.5771 - val_loss: 3.3106 - val_acc: 0.4237
Epoch 8/10
7722/7722 [==============================] - ETA: 0s - loss: 1.7677 - acc: 0.59